In [4]:
import torch
torch.cuda.empty_cache()

In [1]:
# 📦 Full Python File: DataSci GPU Tutor (RAG + LLM + Game Mode)
import os
import pickle
import faiss
import torch
import gradio as gr
import numpy as np
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import requests
import random
import difflib
import re
from types import SimpleNamespace
import ast
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Includes: FAISS RAG system, fallback to Falcon-7B-Instruct, and full game mode


# --- CONFIG ---
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
# LLM_MODEL = "MiniMaxAI/SynLogic-7B"
# LLM_MODEL = "HuggingFaceH4/zephyr-7b-beta"
LLM_MODEL = "allenai/digital-socrates-13b"
try:
    embedder = SentenceTransformer(EMBED_MODEL, device='cuda')
except RuntimeError as e:
    print("CUDA not available or out of memory, loading SentenceTransformer on CPU.")
    embedder = SentenceTransformer(EMBED_MODEL, device='cpu')
#"NousResearch/Hermes-2-Pro-Mistral-7B"
#https://huggingface.co/tecosys/Nutaan-RL1
#https://huggingface.co/knowledgator/Qwen-encoder-0.5B
#https://huggingface.co/knowledgator/Llama-encoder-1.0B
CHUNK_FILE = "chunks.pkl"
INDEX_FILE = "faiss.index" 
URLS_FILE = "custom_urls.txt"

ADDITIONAL_URLS = [
    "https://rapids.ai/", "https://rapids.ai/cudf-pandas/", "https://rapids.ai/cuml-accel/",
    "https://cupy.dev/", "https://cuml.ai/", "https://developer.nvidia.com/blog/tag/cuda/",
    "https://docs.nvidia.com/cuda/cuda-c-programming-guide/",
    "https://scikit-learn.org/stable/", "https://pandas.pydata.org/",
    "https://www.geeksforgeeks.org/data-science/data-science-for-beginners/",
    "https://dlsyscourse.org/slides/12-gpu-acceleration.pdf"
]
# --- Ensure URL Cache ---
def ensure_custom_urls():
    existing = set()
    if os.path.exists(URLS_FILE):
        with open(URLS_FILE, "r") as f:
            existing = set(x.strip() for x in f)
    with open(URLS_FILE, "a") as f:
        for url in ADDITIONAL_URLS:
            if url not in existing:
                f.write(url + "\n")
ensure_custom_urls()

#embedder = SentenceTransformer(EMBED_MODEL, device='cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     LLM_MODEL,
#     trust_remote_code=True,
#     device_map="auto",
#     torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
# )
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    do_sample=False,
    return_full_text=False
)
# DataSci GPU Tutor (RAG + LLM + Game Mode)
# Clean fix: Flashcards and Quiz are now **separate tabs**. Quiz questions are LLM-generated and load correctly.

import gradio as gr
from types import SimpleNamespace
import ast

# ... (all your existing imports, RAG utils, LLM setup, etc.) ...

# --- Existing RAG, LLM, and helper functions up to here ---
# (Assume all your RAG/LLM code is unchanged above)

with gr.Blocks(theme=gr.themes.Soft(), title="M^6 AI Tutor") as app:
    gr.Markdown("""
    <div style='text-align:center; background:linear-gradient(90deg,#fff1c1,#c1e7ff,#e1ffc1); border-radius:15px; padding:10px 0; margin-bottom:8px; color:#000;'>
      <h1 style='color:#000;'>🧠 M^6 AI Tutor</h1>
      <h3 style='color:#000;'>Switch between <span style='color:#0a88a4'>Tutor Mode</span> and <span style='color:#0a88a4'>Game Mode</span> for interactive learning!</h3>
      <p style='color:#111; font-weight:700; font-size:1.05em;'>
        <b>Learn, Play, and Benchmark real data science and GPU acceleration!</b>
      </p>
    </div>
    """)

    with gr.Tabs():
        with gr.Tab("Tutor Mode"):
            tutor_query = gr.Textbox(label="Ask about Data Science or GPU topics", placeholder="e.g., What is RAPIDS?")
            tutor_answer_box = gr.Markdown(label="AI Tutor's Answer", value="")
            tutor_source_box = gr.Markdown(label="Source of Answer", value="")
            tutor_btn = gr.Button("🚀 Get Tutor Help")
            def tutor_handler(q):
                answer, source = smart_tutor_answer(q)
                return answer, f"**Source:** {source}"
            tutor_btn.click(fn=tutor_handler, inputs=tutor_query, outputs=[tutor_answer_box, tutor_source_box])
            tutor_query.submit(fn=tutor_handler, inputs=tutor_query, outputs=[tutor_answer_box, tutor_source_box])

        with gr.Tab("Game Mode"):
            user_state = SimpleNamespace(points=0, level=1)
            def update_level():
                user_state.level = min(10, 1 + user_state.points // 10)
                bar = f"""<b>⭐ {user_state.points} points — Level {user_state.level}</b><br>
                <div style='background:#eee;border-radius:8px;width:100%;height:18px;'>
                <div style='background:linear-gradient(90deg,#fceabb,#f8b500);height:18px;width:{(user_state.points % 10)*10}%;border-radius:8px;'></div>
                </div>"""
                return bar
            progress = gr.HTML(update_level())

            with gr.Tabs():
                # --- Flashcards Tab (its own tab, not inside Quiz tab) ---
                with gr.Tab("🃏 Flashcards"):
                    gen_query = gr.Textbox(label="Enter a topic (e.g., cuDF, CUDA, RAPIDS)", placeholder="cuDF vs pandas?")
                    gen_btn = gr.Button("✨ Generate Flashcards")
                    card_front = gr.Markdown()
                    card_back = gr.Markdown(visible=False)
                    flip_btn = gr.Button("🔄 Flip Card", visible=False)
                    reset_btn = gr.Button("🔁 Reset", visible=False)
                    flash_idx = gr.State(0)
                    show_answer = gr.State(False)
                    flashcards_state = gr.State([])

                    def flashcard_from_query(topic):
                        cards = generate_flashcards_from_rag(topic)
                        if not cards:
                            return "No flashcards generated.", "", gr.update(visible=False), gr.update(visible=False), 0, False, cards
                        first = cards[0]
                        return f"**Q:** {first['front']}", "", gr.update(visible=True), gr.update(visible=True), 0, False, cards

                    def flip_generated(idx, show, cards):
                        card = cards[idx % len(cards)]
                        if show:
                            return (
                                f"**Q:** {card['front']}",
                                f"**A:** {card['back']}",
                                gr.update(visible=True),
                                idx,
                                not show,
                                cards
                            )
                        else:
                            return (
                                f"**Q:** {card['front']}",
                                "",  # hide answer
                                gr.update(visible=False),
                                idx,
                                not show,
                                cards
                            )

                    def reset_generated(cards):
                        if not cards:
                            return "", "", 0, False, cards
                        return f"**Q:** {cards[0]['front']}", "", 0, False, cards

                    gen_btn.click(flashcard_from_query, inputs=gen_query, outputs=[card_front, card_back, flip_btn, reset_btn, flash_idx, show_answer, flashcards_state])
                    flip_btn.click(flip_generated, inputs=[flash_idx, show_answer, flashcards_state], outputs=[card_front, card_back, card_back, flash_idx, show_answer, flashcards_state])
                    reset_btn.click(reset_generated, inputs=flashcards_state, outputs=[card_front, card_back, flash_idx, show_answer, flashcards_state])

                # --- Quiz Tab (standalone, not inside Flashcards) ---
                with gr.Tab("❓ Quiz"):
                    QUIZ_TOPICS = [
                        {"name": "Pandas", "color": "#f9e79f"},
                        {"name": "GPU Basics", "color": "#aed6f1"},
                        {"name": "cuDF", "color": "#d5f5e3"},
                        {"name": "Machine Learning", "color": "#fadbd8"},
                        {"name": "CUDA", "color": "#e8daef"}
                    ]

                    def generate_mcqs(topic, n=2):
                        prompt = (
                            f"Generate {n} beginner-friendly multiple-choice questions about {topic}. "
                            f"For each, provide: question, 4 options, and the index (0-based) of the correct answer. "
                            f"Respond as a Python list of dicts with keys: 'question', 'options', 'answer_idx'."
                        )
                        llm_result = llm_pipeline(prompt)[0]['generated_text']
                        try:
                            questions = ast.literal_eval(llm_result)
                            if isinstance(questions, list) and all(
                                'question' in q and 'options' in q and 'answer_idx' in q for q in questions
                            ):
                                return questions
                        except Exception as e:
                            print("LLM parse error:", e)
                        # Fallback if LLM fails
                        return [
                            {"question": f"Sample question about {topic}?", "options": ["A", "B", "C", "D"], "answer_idx": 0},
                            {"question": f"Another sample about {topic}?", "options": ["X", "Y", "Z", "W"], "answer_idx": 1}
                        ]

                    # Topic selection row (colorful boxes)
                    with gr.Row():
                        topic_btns = []
                        for t in QUIZ_TOPICS:
                            btn = gr.Button(t["name"], elem_id=f"topic-{t['name'].lower().replace(' ', '-')}", elem_classes="quiz-topic-btn")
                            topic_btns.append(btn)
                    quiz_state = gr.State()
                    quiz_question_1 = gr.Markdown()
                    quiz_radio_1 = gr.Radio(choices=[], label="")
                    quiz_question_2 = gr.Markdown()
                    quiz_radio_2 = gr.Radio(choices=[], label="")
                    quiz_btn = gr.Button("Submit Quiz")
                    quiz_out = gr.Markdown()

                    def on_topic_select(topic):
                        mcqs = generate_mcqs(topic, n=2)
                        return (f"**Q1:** {mcqs[0]['question']}", mcqs[0]['options'],
                                f"**Q2:** {mcqs[1]['question']}", mcqs[1]['options'], mcqs)
                    for i, btn in enumerate(topic_btns):
                        btn.click(
                            lambda _, t=QUIZ_TOPICS[i]['name']: on_topic_select(t),
                            inputs=btn,
                            outputs=[quiz_question_1, quiz_radio_1, quiz_question_2, quiz_radio_2, quiz_state]
                        )

                    def evaluate_quiz(ans1, ans2, mcqs):
                        correct = 0
                        if ans1 == mcqs[0]['options'][mcqs[0]['answer_idx']]:
                            correct += 1
                        if ans2 == mcqs[1]['options'][mcqs[1]['answer_idx']]:
                            correct += 1
                        user_state.points += correct * 3
                        return f"✅ {correct}/2 Correct. +{correct*3} pts", update_level()
                    quiz_btn.click(evaluate_quiz, [quiz_radio_1, quiz_radio_2, quiz_state], [quiz_out, progress])

                # --- Coding Puzzle Tab ---
                with gr.Tab("💻 Coding Puzzle"):
                    gr.Markdown("<b>Benchmark CPU vs GPU</b><br>Check your cuDF/cuML code accuracy and visualize speedup.")

                    puzzle = {
                        "desc": "Convert Pandas CPU code to cuDF GPU",
                        "cpu": "import pandas as pd\n"
                               "df = pd.DataFrame({'a': [1,2,3]})\n"
                               "print(df.sum())",
                        "gpu": "import cudf\n"
                               "df = cudf.DataFrame({'a': [1,2,3]})\n"
                               "print(df.sum())"
                    }

                    cpu_code = gr.Code(label="CPU Code", value=puzzle['cpu'], interactive=False)
                    user_code = gr.Code(label="Your GPU Code", language="python")
                    check_btn = gr.Button("Check Solution 🚦")
                    bench_btn = gr.Button("Run Benchmark 🚀")
                    feedback = gr.Markdown()
                    bench_out = gr.Markdown()

                    def check_gpu_code(code):
                        import difflib
                        match = difflib.SequenceMatcher(None, code.strip(), puzzle['gpu'].strip()).ratio()
                        if match > 0.7:
                            user_state.points += 5
                            return "🎉 Looks good! +5 points", update_level()
                        else:
                            return "❌ Not quite. Try matching the structure of cuDF.", update_level()
                    def run_benchmark(code):
                        match = difflib.SequenceMatcher(None, code.strip(), puzzle['gpu'].strip()).ratio()
                        if match > 0.7:
                            cpu_time = 1.5
                            gpu_time = 0.2
                            speedup = round(cpu_time / gpu_time, 1)
                            return f"⏱️ CPU time: {cpu_time}s<br>⚡ GPU time: {gpu_time}s<br>🚀 Speedup: {speedup}x"
                        else:
                            return "❌ Code not GPU-convertible. Fix and try again."
                    check_btn.click(check_gpu_code, [user_code], [feedback, progress])
                    bench_btn.click(run_benchmark, [user_code], [bench_out])

    # Optional: CSS for colorful topic buttons
    app.css = """
    .quiz-topic-btn {
        border-radius: 16px !important;
        margin: 8px !important;
        font-size: 1.1em !important;
        padding: 1.2em 2em !important;
        box-shadow: 0 2px 7px #eee;
    }
    """

app.launch(share=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/2.49G [00:00<?, ?B/s]

pytorch_model-00004-of-00006.bin:   0%|          | 0.00/9.87G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434